In [2]:
#Import Modules

#numpy
import numpy as np
from numpy.random import multivariate_normal #For later example

import pandas as pd

#advectionGP
from advectionGP.models import AdjointAdvectionDiffusionReactionModel as PDEModel #Model module builds basic parts of the PDE problem, combines other classes into full model
from advectionGP.models import AdjointSimpleODEModel as ODEModel #Model module builds basic pa
from advectionGP.sensors import FixedSensorModel #Builds sensor arrays to generate data for foward model or to generate observations for comparison
from advectionGP.kernels import EQ #Generates exponentiated quadratic kernel approximation
from advectionGP.test import TestKernels #Unit test model
from advectionGP.wind import WindFixU #Wind model

#Plotting tools
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

# GPy: Gaussian processes library
import GPy

import scipy

#GPyOpt
import GPyOpt
from GPyOpt.methods import BayesianOptimization

In [3]:
# generate sensor locations for training data with shape [total observations, 4], where each row has elements 
#[lower time location, upper time location, x location, y location]

tlocL = np.linspace(1,8,4) # lower time
xloc=np.linspace(2,8,5) # x locations
yloc=np.linspace(2,8,5) # y locations
sensN = len(xloc)*len(yloc) # total number of sensors 
obsN = len(tlocL) # total time points at which an observation is taken
X= np.zeros((obsN*sensN,4)) # obsN*sensN is total observations over all sensors and all times
# Build sensor locations
X[:,0] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[0] #lower time
X[:,2] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[1] # x location
X[:,3] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[2] # ylocation
X[:,1] = X[:,0]+1 # upper time

XGP= np.zeros((sensN*obsN,3))
XGP[:,0] = (X[:,0]+X[:,1])/2
XGP[:,1] = X[:,2]
XGP[:,2] = X[:,3] 



sensors = FixedSensorModel(X,1) # establish sensor model arguments are sensor locations and spatial averaging

# generate sensor locations for test data with shape [total observations, 4], where each row has elements 
#[lower time location, upper time location, x location, y location]

tlocL = np.linspace(2,9,5) # lower time
xloc=np.linspace(1.5,8.5,4) # x locations
yloc=np.linspace(1.5,8.5,4) # y locations
sensN = len(xloc)*len(yloc) # total number of sensors 
obsN = len(tlocL) # total time points at which an observation is taken
Xtest= np.zeros((obsN*sensN,4)) # obsN*sensN is total observations over all sensors and all times
# Build sensor locations
Xtest[:,0] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[0] 
Xtest[:,2] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[1]
Xtest[:,3] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[2]
Xtest[:,1] = Xtest[:,0]+1

XGPtest= np.zeros((sensN*obsN,3))
XGPtest[:,0] = (Xtest[:,0]+Xtest[:,1])/2
XGPtest[:,1] = Xtest[:,2]
XGPtest[:,2] = Xtest[:,3]

sensorsTest = FixedSensorModel(Xtest,1) # establish sensor model

In [4]:
k_0 = 0.01 #Diffusion
R=0.1
noiseSD = 0.05 #Observation noise
N_feat=2000 # number of features used to approximate ground truth GP
boundary = ([0,0,0],[10,10,10])# corners of the grid - in units of space
k = EQ(2, 2.0) # generate EQ kernel arguments are lengthscale and variance
res = [50,30,30] # grid size for time, x and y
wind=np.cos(np.linspace(0,6*np.pi,res[1]))*0.5
u=[]
u.append(np.ones(res)*0.01) #x direction wind
u.append(np.ones(res)*0.01) # y direction wind
windmodel=WindFixU(u)
#u.append(np.ones(res)*0.1) #x direction wind
#u.append(np.ones(res)*0.1) # y direction wind
m = PDEModel(resolution=res,boundary=boundary,N_feat=N_feat,noiseSD=noiseSD,kernel=k,sensormodel=sensors,windmodel=windmodel,k_0=k_0,R=R) #initiate PDE model to build concentration

dt,dx,dy,dx2,dy2,Nt,Nx,Ny = m.getGridStepSize() # useful numbers!

z=np.random.normal(0,1.0,N_feat) # Generate z to compute source
sourceGT=(m.computeSourceFromPhi(z))# Compute source

In [5]:
concTrain=m.computeConcentration(sourceGT) # Compute concentration - runs advection diffusion forward model
yTrain= m.computeObservations() # Compute observations with noise uses m.sensormodel for observation locations

m.sensormodel=sensorsTest
yTest= m.computeObservations() 

In [6]:
k2 = EQ(2, 2.0) 

In [7]:
mPDE = PDEModel(resolution=res,boundary=boundary,N_feat=100,noiseSD=noiseSD,kernel=k2,sensormodel=sensors,windmodel=windmodel,k_0=k_0,R=R) #Initiate new model for inference


In [8]:
%%timeit -r 4 -n 10
regressPDE = mPDE.computeModelRegressors() # Compute regressor matrix
meanZPDE, covZPDE = mPDE.computeZDistribution(yTrain) # Infers z vector mean and covariance
sourcePDE = mPDE.computeSourceFromPhi(meanZPDE) # Generates estimated source using inferred distributio

Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculat

In [10]:
import time
total_its =[]

In [15]:
mMCMC = PDEModel(resolution=res,boundary=boundary,N_feat=N_feat,noiseSD=noiseSD,kernel=k2,sensormodel=sensors,windmodel=windmodel,k_0=k_0,R=R) #initiate PDE model to build concentration

In [24]:
%%timeit -r 2 -n 5
N_feat = 100
k2 = EQ(2, 2.0)
sigmaMCMC=0.005
out2 = []
var = np.ones(len(yTrain))*noiseSD**2
accept = 0 
start_time2 = time.time()
t_end = time.time() +1.19
iters = 0
ObsN=75
#mMCMC = PDEModel(resolution=res,boundary=boundary,N_feat=N_feat,noiseSD=noiseSD,kernel=k2,sensormodel=sensors,windmodel=windmodel,k_0=k_0,R=R) #initiate PDE model to build concentration
#zCur=np.random.normal(0,1.0,N_feat) # Generate z to compute source
zCur=np.ones(N_feat)


source = mMCMC.computeSourceFromPhi(zCur) # Generates estimated source using inferred distribution

conc=mMCMC.computeConcentration(source) # Compute test concentration

yMCMC= mMCMC.computeObservations() # Compute observations with noise
#llObsCur = -0.5*np.sum((np.square(np.subtract(yTrain,yMCMC))/(noiseSD**2)))-0.5*ObsN*np.log(noiseSD**2)-0.5*ObsN*np.log(2*np.pi)
logPiCur = -np.inf
llObsCur = 0
iterations=20000
#out2.append(zCur)
#for zi in zCur:
#    logPriorCur += np.log(scipy.stats.norm(0, 1).pdf(zi))

for i in range(iterations):
    zCan = zCur + np.random.normal(0,sigmaMCMC,N_feat)
    time1 = time.time()
    source = mMCMC.computeSourceFromPhi(zCan) # Generates estimated source using inferred distribution
    print(time.time()-time1,"source calc")
    
    time2 = time.time()
    concCan=mMCMC.computeConcentration(source) # Compute test concentration
    print(time.time()-time2,"conc calc")
    
    time3 = time.time()
    
    yCan= mMCMC.computeObservations() # Compute observations with noise
    print(time.time()-time3,"obs calc")
    
    
    llObsCan = -0.5*np.sum((np.square(np.subtract(yTrain,yCan))/(noiseSD**2)))-0.5*ObsN*np.log(noiseSD**2)-0.5*ObsN*np.log(2*np.pi)
    logPriorCan = 0
    for j in zCan:
        logPriorCan += np.log(scipy.stats.norm(0, 1).pdf(j))

    logPiCan = llObsCan + logPriorCan
    #logPiCur = llObsCur + logPriorCur
    #print(logPiCan,logPiCur)
    u = np.random.uniform(0,1)

    if np.log(u) < logPiCan - logPiCur:
        zCur = zCan
        #logPriorCur = logPriorCan
        logPiCur = logPiCan
        accept += 1###
    out2.append(zCur)
    iters +=1
    if time.time() > t_end:
        break 
total_its.append(np.array(iters))        
print(iters)


0.0568695068359375 source calc
0.018981456756591797 conc calc
0.010876655578613281 obs calc
0.0443272590637207 source calc
0.008962154388427734 conc calc
0.008363008499145508 obs calc
0.05272269248962402 source calc
0.008682489395141602 conc calc
0.007318258285522461 obs calc
0.05886483192443848 source calc
0.009906530380249023 conc calc
0.0060575008392333984 obs calc
0.05488991737365723 source calc
0.01698017120361328 conc calc
0.012916326522827148 obs calc
0.06368136405944824 source calc
0.007973194122314453 conc calc
0.011046886444091797 obs calc
0.05659651756286621 source calc
0.008970022201538086 conc calc
0.006951570510864258 obs calc
7
0.06619477272033691 source calc
0.013601064682006836 conc calc
0.010906457901000977 obs calc
0.06310057640075684 source calc
0.009366512298583984 conc calc
0.014089107513427734 obs calc
0.061446189880371094 source calc
0.009963512420654297 conc calc
0.01290130615234375 obs calc
0.07752084732055664 source calc
0.0101470947265625 conc calc
0.0079729

In [25]:
np.mean(total_its)

10.173913043478262

In [26]:
np.std(total_its)

5.543520886451635

# Timing ODE

In [27]:
#numpy
import numpy as np
from numpy.random import multivariate_normal #For later example

#advectionGP
from advectionGP.models import AdjointSecondOrderODEModel as Model #Model module builds basic parts of the ODE problem, combines other classes into full model

'''To use other models in the class, such as the Advection Diffusion Reaction model, 
replace "AdjointSecondOrderODEModel" in the line above with "AdjointAdvectionDiffusionReaction" model
and adjust parameters as appropriate'''

from advectionGP.sensors import FixedSensorModel #Builds sensor arrays to generate data for foward model or to generate observations for comparison
from advectionGP.kernels import EQ #Generates exponentiated quadratic kernel approximation
from advectionGP.test import TestKernels #Unit test model


#Plotting tools
import matplotlib.pyplot as plt

In [28]:
avg=0.1 # time that an observation is taken for
tlocL = np.linspace(0,9.9,100) # observation start times
X= np.zeros((len(tlocL),2)) # initiate X
# Build sensor locations
X[:,0] = tlocL #lower time
X[:,1] = X[:,0]+avg # upper time

In [29]:
sensors = FixedSensorModel(X,0) # establish sensor model arguments are sensor locations and spatial averaging (not applicable in 1D case so set to 0)

In [30]:
k_0 = -0.5 #Diffusion coefficient
u=1
eta=5
noiseSD = 0.05 #Observation noise
N_feat=2000 # number of features used to approximate ground truth GP
boundary = ([0],[10])# edges of the grid - in units of time
kForward = EQ(0.6, 4.0) # generate EQ kernel arguments are lengthscale and variance
res = [100] # grid size for time

In [31]:
m = Model(resolution=res,boundary=boundary,N_feat=N_feat,noiseSD=noiseSD,kernel=kForward,sensormodel=sensors,k_0=k_0,u=u,eta=eta) #initiate PDE model to build concentration

dt,dt2,Nt = m.getGridStepSize() # useful numbers!

z=np.random.normal(0,1.0,N_feat) # Generate z to compute source
sourceGT=m.computeSourceFromPhi(z)# Compute ground truth source by approximating GP
#sourceGT = np.ones(m.resolution)  # Example constant source
concTrain=m.computeConcentration(sourceGT) # Compute concentration - runs ODE forward model
yTrain= m.computeObservations() # Compute observations with noise uses m.sensormodel for observation locations

In [32]:
%%timeit -r 4 -n 10
N_feat =100 #Number of features used to infer the source
kInverse = EQ(0.6, 4) # Initiate kernel for inverse problem
mInfer = Model(resolution=res,boundary=boundary,N_feat=N_feat,noiseSD=noiseSD,kernel=kInverse,sensormodel=sensors,k_0=k_0,u=u,eta=eta) #Initiate new model for inference
mInfer.computeModelRegressors() # Compute regressor matrix 
meanZ, covZ = mInfer.computeZDistribution(yTrain) # Infers z vector mean and covariance using regressor matrix
sourceInfer = mInfer.computeSourceFromPhi(meanZ) # Generates estimated source using mean of the inferred distribution
concInfer=m.computeConcentration(sourceInfer) # Generates estimated concentration from inferred source


Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculating Phis...
99/100 
Calculating Adjoints...
99/100 
Calculat

In [33]:
import time
import scipy.stats

In [34]:
for i in range(100):
    k2 = EQ(2, 2.0)
    sigmaMCMC=0.005
    out2 = []
    var = np.ones(len(yTrain))*noiseSD**2
    accept = 0 
    start_time2 = time.time()
    t_end = time.time() +0.562
    iters = 0
    ObsN=50

    mMCMC = Model(resolution=res,boundary=boundary,N_feat=N_feat,noiseSD=noiseSD,kernel=k2,sensormodel=sensors,k_0=k_0,u=u,eta=eta) #Initiate new model for inference
    #zCur=np.random.normal(0,1.0,N_feat) # Generate z to compute source
    zCur=np.ones(N_feat)
    source = mMCMC.computeSourceFromPhi(zCur) # Generates estimated source using inferred distribution
    conc=mMCMC.computeConcentration(source) # Compute test concentration
    yMCMC= mMCMC.computeObservations() # Compute observations with noise
    #llObsCur = -0.5*np.sum((np.square(np.subtract(yTrain,yMCMC))/(noiseSD**2)))-0.5*ObsN*np.log(noiseSD**2)-0.5*ObsN*np.log(2*np.pi)
    logPiCur = -np.inf
    llObsCur = 0
    iterations=20000
    #out2.append(zCur)
    #for zi in zCur:
    #    logPriorCur += np.log(scipy.stats.norm(0, 1).pdf(zi))

    for i in range(iterations):
        zCan = zCur + np.random.normal(0,sigmaMCMC,N_feat)
        time1 = time.time()
        source = mMCMC.computeSourceFromPhi(zCan) # Generates estimated source using inferred distribution
        print(time.time()-time1,"source calc")

        time2 = time.time()
        concCan=mMCMC.computeConcentration(source) # Compute test concentration
        print(time.time()-time2,"conc calc")

        time3 = time.time()

        yCan= mMCMC.computeObservations() # Compute observations with noise
        print(time.time()-time3,"obs calc")
        llObsCan = -0.5*np.sum((np.square(np.subtract(yTrain,yCan))/(noiseSD**2)))-0.5*ObsN*np.log(noiseSD**2)-0.5*ObsN*np.log(2*np.pi)
        logPriorCan = 0
        for j in zCan:
            logPriorCan += np.log(scipy.stats.norm(0, 1).pdf(j))

        logPiCan = llObsCan + logPriorCan
        #logPiCur = llObsCur + logPriorCur
        #print(logPiCan,logPiCur)
        u = np.random.uniform(0,1)

        if np.log(u) < logPiCan - logPiCur:
            zCur = zCan
            #logPriorCur = logPriorCan
            logPiCur = logPiCan
            accept += 1
        out2.append(zCur)
        iters +=1
        if time.time() > t_end:
            break 

    print(iters)

0.028262615203857422 source calc
0.0016779899597167969 conc calc
0.0026657581329345703 obs calc
1
0.024389982223510742 source calc
0.000995635986328125 conc calc
0.000997304916381836 obs calc
1
0.017940521240234375 source calc
0.0009968280792236328 conc calc
0.001989603042602539 obs calc
1
0.01793956756591797 source calc
0.0009965896606445312 conc calc
0.0009975433349609375 obs calc
1
0.018939495086669922 source calc
0.0019903182983398438 conc calc
0.0009968280792236328 obs calc
1
0.029171228408813477 source calc
0.001031637191772461 conc calc
0.001956462860107422 obs calc
1
0.024918317794799805 source calc
0.0009968280792236328 conc calc
0.0019941329956054688 obs calc
1
0.0238649845123291 source calc
0.000997304916381836 conc calc
0.0019958019256591797 obs calc
1
0.017069578170776367 source calc
0.0009963512420654297 conc calc
0.0009968280792236328 obs calc
1
0.019935131072998047 source calc
0.000997304916381836 conc calc
0.0009963512420654297 obs calc
1
0.02286982536315918 source cal

1
0.04550528526306152 source calc
0.0027468204498291016 conc calc
0.0045740604400634766 obs calc
1
0.02716684341430664 source calc
0.0009758472442626953 conc calc
0.0020151138305664062 obs calc
1
0.030931472778320312 source calc
0.0009961128234863281 conc calc
0.0020246505737304688 obs calc
1
0.025865793228149414 source calc
0.0010077953338623047 conc calc
0.0019898414611816406 obs calc
1
0.028142690658569336 source calc
0.001984834671020508 conc calc
0.0009386539459228516 obs calc
1
0.0189359188079834 source calc
0.0009968280792236328 conc calc
0.0019931793212890625 obs calc
1
0.027887821197509766 source calc
0.0010216236114501953 conc calc
0.0019712448120117188 obs calc
1
0.03102564811706543 source calc
0.0019500255584716797 conc calc
0.0009982585906982422 obs calc
1
0.02591419219970703 source calc
0.0010058879852294922 conc calc
0.0019888877868652344 obs calc
1
0.026203155517578125 source calc
0.000997304916381836 conc calc
0.0010216236114501953 obs calc
1
0.0258944034576416 source 